In [ ]:
# Install llama.cpp
!git clone https://github.com/ggerganov/llama.cpp
!cd llama.cpp && git pull && make clean && LLAMA_CUBLAS=1 make
!pip install -r llama.cpp/requirements.txt

Cloning into 'llama.cpp'...
remote: Enumerating objects: 12126, done.
remote: Counting objects: 100% (3078/3078), done.
remote: Compressing objects: 100% (206/206), done.
remote: Total 12126 (delta 2958), reused 2887 (delta 2872), pack-reused 9048
Receiving objects: 100% (12126/12126), 14.52 MiB | 5.66 MiB/s, done.
Resolving deltas: 100% (8430/8430), done.
Already up to date.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -Wdouble-promotion -pthread -march=native -mtune=native 
I CXXFLAGS:  -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread  -Wno-

In [ ]:
# Variables
MODEL_ID = "Manirajan/llama2-robot_action"
QUANTIZATION_METHODS = ["q4_k_m"]

# Constants
MODEL_NAME = MODEL_ID.split('/')[-1]
GGML_VERSION = "gguf"

In [ ]:
# Download model
!git lfs install
!git clone https://huggingface.co/{MODEL_ID}

Git LFS initialized.
Cloning into 'llama2-robot_action'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 15 (delta 1), reused 0 (delta 0), pack-reused 3
Unpacking objects: 100% (15/15), 479.67 KiB | 2.25 MiB/s, done.
Filtering content: 100% (2/2), 4.55 GiB | 7.22 MiB/s, done.
Encountered 1 file(s) that may not have been copied correctly on Windows:
	pytorch_model-00001-of-00002.bin

See: `git lfs help smudge` for more details.


In [ ]:
# Convert to fp16
fp16 = f"{MODEL_NAME}/{MODEL_NAME.lower()}.{GGML_VERSION}.fp16.bin"
!python llama.cpp/convert.py {MODEL_NAME} --outtype f16 --outfile {fp16}


Loading model file llama2-robot_action/pytorch_model-00001-of-00002.bin
Loading model file llama2-robot_action/pytorch_model-00001-of-00002.bin
Loading model file llama2-robot_action/pytorch_model-00002-of-00002.bin
params = Params(n_vocab=32000, n_embd=4096, n_layer=32, n_ctx=4096, n_ff=11008, n_head=32, n_head_kv=32, f_norm_eps=1e-05, rope_scaling_type=None, f_rope_freq_base=None, f_rope_scale=None, n_orig_ctx=None, rope_finetuned=None, ftype=<GGMLFileType.MostlyF16: 1>, path_model=PosixPath('llama2-robot_action'))
Loading vocab file 'llama2-robot_action/tokenizer.model', type 'spm'
Permuting layer 0
Permuting layer 1
Permuting layer 2
Permuting layer 3
Permuting layer 4
Permuting layer 5
Permuting layer 6
Permuting layer 7
Permuting layer 8
Permuting layer 9
Permuting layer 10
Permuting layer 11
Permuting layer 12
Permuting layer 13
Permuting layer 14
Permuting layer 15
Permuting layer 16
Permuting layer 17
Permuting layer 18
Permuting layer 19
Permuting layer 20
Permuting layer 21


In [ ]:

# Quantize the model for each method in the QUANTIZATION_METHODS list
for method in QUANTIZATION_METHODS:
    qtype = f"{MODEL_NAME}/{MODEL_NAME.lower()}.{GGML_VERSION}.{method}.bin"
    !./llama.cpp/quantize {fp16} {qtype} {method}

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5
main: build = 1523 (947f64f)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing 'llama2-robot_action/llama2-robot_action.gguf.fp16.bin' to 'llama2-robot_action/llama2-robot_action.gguf.q4_k_m.bin' as Q4_K_M
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from llama2-robot_action/llama2-robot_action.gguf.fp16.bin (version GGUF V3 (latest))
llama_model_loader: - tensor    0:                token_embd.weight f16      [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight f16      [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight f16      [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight f16      [  4096,  

In [ ]:
import os

model_list = [file for file in os.listdir(MODEL_NAME) if GGML_VERSION in file]

prompt = input("Input")
chosen_method = input("Please specify the quantization method to run the model (options: " + ", ".join(model_list) + "): ")

# Verify the chosen method is in the list
if chosen_method not in model_list:
    print("Invalid method chosen!")
else:
    qtype = f"{MODEL_NAME}/{MODEL_NAME.lower()}.{GGML_VERSION}.{method}.bin"
    !./llama.cpp/main -m {qtype} -n 128 --color -ngl 35 -p "{prompt}"

InputBelow is the instruction that describes a task paired with input that provides further context. Write a response that appropriately completes the request\nn### Instruction:\nAdjust all joints to a custom pose, then move to the box and put it in the specified container.\n ###Human: \n generate CIL (high level robotic commands) for the prompt Adjust all joints to a custom pose, then move to the box and put it in the specified container 
Please specify the quantization method to run the model (options: llama2-robot_action.gguf.fp16.bin, llama2-robot_action.gguf.q4_k_m.bin): llama2-robot_action.gguf.q4_k_m.bin
Log start
main: build = 1523 (947f64f)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: seed  = 1700229484
ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5
llama_model_loader: loaded meta data with 20 key-value pairs and

In [ ]:
!pip install -q huggingface_hub

username = "Manirajan"

from huggingface_hub import notebook_login, create_repo, HfApi
notebook_login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 5.9 MB/s eta 0:00:00


In [ ]:
api = HfApi()


create_repo(repo_id = f"{username}/{MODEL_NAME}-GGUF", repo_type="model", exist_ok=True)

api.upload_folder(
    folder_path=MODEL_NAME,
    repo_id=f"{username}/{MODEL_NAME}-GGUF",
    allow_patterns=f"*{GGML_VERSION}*",
)

llama2-robot_action.gguf.q4_k_m.bin:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

llama2-robot_action.gguf.fp16.bin:   0%|          | 0.00/13.5G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

'https://huggingface.co/Manirajan/llama2-robot_action-GGUF/tree/main/'